In [1]:
import numpy as np
#import sys, os
#sys.path.append(os.path.abspath('../../src/neural_network'))
#sys.path.append('../../src/neural_network')
from neural_network import NeuralNetwork
from fc_layer import FCLayer
from activation_layer import ActivationLayer
from activation_functions import tanh, tanh_derivative, relu, relu_derivative, softmax, softmax_derivative
from loss_functions import mse, mse_derivative, cross_entropy, cross_entropy_derivative
import ray
from ray import tune
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.suggest.bayesopt import BayesOptSearch
import sklearn
from sklearn import *

In [2]:
X, Y = datasets.load_wine(return_X_y=True)
print(X.shape, Y.shape)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,Y, test_size=0.2)
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(178, 13) (178,)
(142, 1, 13) (36, 1, 13) (142,) (36,)


In [4]:
network = NeuralNetwork()
network.add(FCLayer(X_train.shape[2], 5))
network.add(ActivationLayer(softmax, softmax_derivative))
network.add(FCLayer(5, 2))
network.add(ActivationLayer(softmax, softmax_derivative))
network.add(FCLayer(2, 3))
network.add(ActivationLayer(softmax, softmax_derivative))
network.use(cross_entropy, cross_entropy_derivative)
network.fit(X_train, y_train, epochs=100, learning_rate=0.001)
y_pred = np.array(network.predict(X_test))
print(y_pred, y_pred.shape)
y_pred = y_pred.reshape(y_pred.shape[0],)
print(sklearn.metrics.accuracy_score(y_test, y_pred))

[[[nan]]

 [[nan]]

 [[nan]]

 [[nan]]

 [[nan]]

 [[nan]]

 [[nan]]

 [[nan]]

 [[nan]]

 [[nan]]

 [[nan]]

 [[nan]]

 [[nan]]

 [[nan]]

 [[nan]]

 [[nan]]

 [[nan]]

 [[nan]]

 [[nan]]

 [[nan]]

 [[nan]]

 [[nan]]

 [[nan]]

 [[nan]]

 [[nan]]

 [[nan]]

 [[nan]]

 [[nan]]

 [[nan]]

 [[nan]]

 [[nan]]

 [[nan]]

 [[nan]]

 [[nan]]

 [[nan]]

 [[nan]]] (36, 1, 1)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [4]:
def my_func(config, reporter):  # add the reporter parameter
      # report the configuration
    network = NeuralNetwork()
    network.add(FCLayer(X_train.shape[2], 5))
    network.add(ActivationLayer(softmax, softmax_derivative))
    network.add(FCLayer(5, 2))
    network.add(ActivationLayer(softmax, softmax_derivative))
    network.add(FCLayer(2, 3))
    network.add(ActivationLayer(softmax, softmax_derivative))
    network.use(mse, mse_derivative)
    network.fit(X_train, y_train, epochs=100, learning_rate=config["alpha"])
    y_pred = np.array(network.predict(X_test))
    y_pred = y_pred.reshape(y_pred.shape[0],)
    reporter(config, mean_loss = sklearn.metrics.accuracy_score(y_test, y_pred))
    
tune.register_trainable("my_func", my_func)
ray.shutdown()
ray.init()

res = tune.run_experiments({
    "my_experiment": {
        "run": "my_func",
        "stop": { "mean_accuracy": 0.6 },
        "config": {
            "alpha": tune.grid_search([0.1, 0.2, 0.3, 0.4]),
            "hidden_layer_sizes": tune.grid_search([(5,2)]),
        }
    }
})

ModuleNotFoundError: No module named 'gym'